# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

### Spellchecker interface

In [24]:
from typing import NewType

Word = NewType("Word", str)
Sentence = NewType("Sentence", str)

In [46]:
from abc import ABC, abstractmethod


class Spellchecker(ABC):
    vocab: set[Word]
    edit_distance: int

    @abstractmethod
    def correct_sentence(self, sentence: Sentence, skip_in_vocab: bool = True) -> Sentence:
        return NotImplemented

    @staticmethod
    def get_words(text) -> list[Word]:
        return list(Word(w) for w in re.findall(r"\w+", text.lower()))

    @staticmethod
    def edits(word: Word) -> set[Word]:
        letters = "abcdefghijklmnopqrstuvwxyz"
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(Word(w) for w in deletes + transposes + replaces + inserts)

    def edit_up_to_distance(self, word: str) -> list[set[Word]]:
        results = [{word}]
        for _ in range(self.edit_distance):
            edits = set()
            for word in results[-1]:
                edits |= self.edits(word)
            results.append(edits)
        return results

### Prepare data (train/test split)

In [47]:
import nltk
from pathlib import Path
from sklearn.model_selection import train_test_split

sentences = nltk.sent_tokenize(Path("big.txt").read_text())

In [48]:
len(sentences)

57635

In [49]:
train_corpus, test_corpus = train_test_split(sentences, test_size=0.2, random_state=42)

Path("big.train.txt").write_text(" ".join(train_corpus))
Path("big.test.txt").write_text(" ".join(test_corpus))

1258764

### Norvig's solution (baseline) 

In [50]:
from typing import Iterable
import re
from collections import Counter


class SimpleSpellChecker(Spellchecker):
    def __init__(self, train_corpus: Path, edit_distance: int = 2) -> None:
        self.words = Counter(self.get_words(train_corpus.read_text()))
        self.vocab = set(self.words.keys())
        self.word_count = sum(self.words.values())
        self.edit_distance = edit_distance

    def probability(self, word: str) -> float:
        return self.words.get(word, 0) / self.word_count

    def correct_one_word(self, word: str) -> str:
        return max(self.get_candidates(word), key=self.probability)

    def get_candidates(self, word: str) -> set[str]:
        for option in map(self.get_known_words, self.edit_up_to_distance(word)):
            if option:
                return option

        return {word}

    def get_known_words(self, words: Iterable[str]) -> set[str]:
        return set(words).intersection(self.vocab)

    def correct_sentence(self, sentence: Sentence, skip_in_vocab: bool = True) -> Sentence:
        return Sentence(" ".join(self.correct_one_word(w) for w in self.get_words(sentence)))

In [51]:
sp = SimpleSpellChecker(Path("big.train.txt"))

In [52]:
sp.correct_one_word("justificaitons")

'justifications'

In [53]:
sp.correct_sentence("Sherlck Halmes")

'sherlock holmes'

In [54]:
sp.correct_sentence("the way bark")

'the way bark'

In [55]:
sp.word_count

896252

### N-gram spellchecker

In [136]:
from itertools import chain
from collections import defaultdict

NO_WORD = "-"


class NGramSpellChecker(Spellchecker):
    def __init__(
        self, train_corpus: Path, edit_distance: int = 2, ngram_size: int = 2
    ) -> None:
        words = self.get_words(train_corpus.read_text())
        self.vocab = set(words)
        self.word_count = len(words)
        
        self.edit_distance = edit_distance
        self.ngram_counts = Counter(self._get_ngrams(words, ngram_size))
        self.ngram_count = sum(self.ngram_counts.values())
        self.ngram_size = ngram_size
        
        self.prefix = defaultdict(int)
        self.suffix = defaultdict(int)
        
        for ngram, count in self.ngram_counts.items():
            for k in range(1, len(ngram) + 1):
                k_gram = ngram[:k]
                self.prefix[k_gram] += count
                self.suffix[k_gram[::-1]] += count
        self.prefix[tuple()] = self.suffix[tuple()] = self.ngram_count

    @staticmethod
    def _get_ngrams(words: Iterable[str], ngram_size: int, pad: bool = False) -> list[Word]:
        if pad:
            words = chain([NO_WORD] * (ngram_size - 1), words)
        else:
            words = words
        return nltk.ngrams(words, ngram_size)

    def word_probability(self, context_before: tuple[Word, ...], word: Word, context_after: tuple[Word, ...]) -> float:
        before_prob = self.prefix[context_before + (word,)] / self.ngram_count
        after_prob = self.suffix[context_after[::-1] + (word,)] / self.ngram_count
        return before_prob + after_prob

    def correct_word(self, context_before: tuple[Word, ...], word: Word, context_after: tuple[Word, ...]) -> Word:
        return max(
            set().union(*self.edit_up_to_distance(word)), key=lambda w: self.word_probability(context_before, w, context_after)
        )

    def correct_sentence(self, sentence: Sentence, skip_in_vocab: bool = True) -> Sentence:
        words = self.get_words(sentence)
        new_words = []

        for i, word in enumerate(words):
            if skip_in_vocab and word in self.vocab:
                new_words.append(word)
            else:
                context_before = tuple(new_words[-self.ngram_size:])
                context_after = tuple(words[i + 1:i + self.ngram_size])
                new_words.append(self.correct_word(context_before, word, context_after))

        return Sentence(" ".join(new_words))

In [137]:
ngram_sp = NGramSpellChecker(Path("big.train.txt"))

In [138]:
ngram_sp.correct_sentence("Sherlck Halmes")

'sherlock holmes'

In [139]:
ngram_sp.correct_sentence("the way bark", skip_in_vocab=False)

'the war are'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

I used the same train set for both spellcheckers (Norvig's dataset) for more fair comparison. I did not assign any weights to edits and did not use something like beam for simplicity, but adding it may lead to improved accuracy.

For n-gram spellchecker I used bigrams and unigrams, both looking forward and backward in context. Trying to add trigrams and etc. did not improve the performance of the model. I decided to add context after word to improve the accuracy, but it did not seem to help as well. Also, I use already predicted words as the context before word because it yields better results. I tried to predict words even if they are in dictionary, but it leads to bad results, maybe adding weights to edits would help with it.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

### Define test set generation

In [126]:
import random


def mutate_word(word: str, edit_probability: float = 1e-1) -> str:
    """
    1 edit probability - 1/10
    2 edit probability - 1/100
    and etc.
    """
    while random.random() <= edit_probability:
        word = next(iter(Spellchecker.edits(word)))
    return word

In [127]:
test_sentences = nltk.sent_tokenize(Path("big.test.txt").read_text())
test_words = [Spellchecker.get_words(s) for s in test_sentences]

In [128]:
random.seed(42)
mutated_words = [[mutate_word(w) for w in s] for s in test_words]

In [129]:
mutated_words[0]

['by',
 'bmeans',
 'of',
 'two',
 'recurved',
 'fangs',
 'attacned',
 'so',
 'the',
 'umper',
 'jaw',
 'and',
 'connected',
 'by',
 'a',
 'ductv',
 'with',
 'poison',
 'secreting',
 'glands',
 'they',
 'intruduce',
 'into',
 'their',
 'prey',
 'a',
 'thick',
 'transparent',
 'yellowish',
 'fluid',
 'of',
 'acid',
 'reaction',
 'probably',
 'ouf',
 'the',
 'wature',
 'of',
 'an',
 'albumose',
 'and',
 'known',
 'as',
 'the',
 '_venom_']

### Comparing solutions

In [130]:
class DoNothing(Spellchecker):
    def correct_sentence(self, sentence: str) -> str:
        return sentence

In [131]:
import time


def test_spellchecker(sp: Spellchecker, test_set: list[list[str]] = test_words) -> None:
    prepared_data = [" ".join(s) for s in test_set]
    start = time.time()
    all_predictions = []
    for mutated_sentence in prepared_data:
        predicted = sp.correct_sentence(mutated_sentence)
        all_predictions.append(predicted)
    end = time.time()
    all_predictions = [s.split() for s in all_predictions]
    elapsed = end - start
    print(f"Time taken: {elapsed:.3f} s")
    total_words = sum(len(s) for s in all_predictions)
    print("Processed", total_words, "words")
    correct_words = sum(
        w1 == w2
        for s1, s2 in zip(test_words, all_predictions)
        for w1, w2 in zip(s1, s2)
    )
    accuracy = correct_words / total_words
    print(f"Accuracy: {accuracy:.2%}")
    print(f"Rate: {total_words / elapsed:.2f} words/s")

In [132]:
NUM_SENTENCES = 20

In [133]:
test_spellchecker(DoNothing(), mutated_words[:NUM_SENTENCES])

Time taken: 0.000 s
Processed 431 words
Accuracy: 87.01%
Rate: 36892755.59 words/s


In [134]:
test_spellchecker(sp, mutated_words[:NUM_SENTENCES])

Time taken: 11.442 s
Processed 431 words
Accuracy: 93.50%
Rate: 37.67 words/s


In [140]:
test_spellchecker(ngram_sp, mutated_words[:NUM_SENTENCES])

Time taken: 4.772 s
Processed 431 words
Accuracy: 90.02%
Rate: 90.31 words/s


It seems that my implementation perform noticeably worse than Norvig's on such test set. It would be nice to test it on some other dataset, but I cannot find any.
But at least it is significantly faster than Norvig's
 